In [2]:
using Random
using Distributions
using FLoops
using Base.Threads
using BenchmarkTools

include("data.jl")
include("util.jl")

include("intervals/permutation.jl")
include("intervals/bootstrap.jl")
include("intervals/t.jl")

Threads.nthreads()

112

In [3]:
dtype = Float32
seed = 123

# DATA CONFIG

alpha = 0.05

# data
B  = 100   # num. coverage probabilities per boxplot
S  = 4300  # num. samples per coverage probability
nx = 8   # size of group 1
ny = 9    # size of group 2
MC = 10_000
perms = nothing
if binomial(nx + ny, nx) < 30_000
    perms = partition(nx, ny)
end

([1 1 … 9 10; 2 2 … 11 11; … ; 7 7 … 16 16; 8 9 … 17 17], [17 17 … 10 9; 16 16 … 8 8; … ; 10 10 … 2 2; 9 8 … 1 1])

In [4]:
# POPULATION SETTINGS

Random.seed!(123)

distrTypeX = LogNormal{dtype}
X_mu = random(Uniform(0, 1), B)
X_sigma = random(Uniform(0, 0.6), B)
distrX = map(distrTypeX, X_mu, X_sigma)

distrTypeY = Gamma{dtype}
Y_shape = @. 1 / (exp(X_sigma^2) - 1)
Y_scale = @. exp(X_mu + X_sigma^2 / 2) / Y_shape
distrY = map(distrTypeY, Y_shape, Y_scale)
;

In [5]:
deltas = @. mean(distrX) - mean(distrY)

@show distrX[1:2]
@show distrY[1:2]
@show deltas[1:2];

distrX[1:2] = LogNormal{Float32}[LogNormal{Float32}(μ=0.906f0, σ=0.572f0), LogNormal{Float32}(μ=0.443f0, σ=0.508f0)]
distrY[1:2] = Gamma{Float32}[Gamma{Float32}(α=2.5836008f0, θ=1.1279569f0), Gamma{Float32}(α=3.3964891f0, θ=0.5216751f0)]
deltas[1:2] = Float32[0.0, 0.0]


In [6]:
Random.seed!(123)
xs = [dtype.(rand(distrX[i], nx, S)) for i in 1:B]
ys = [dtype.(rand(distrY[i], ny, S)) for i in 1:B]
@show size(ys)
ys[2]

size(ys) = (100,)


9×4300 Matrix{Float32}:
 0.918382  1.17214   1.58792   1.78473   …  1.57903  0.909114  1.3323
 5.10085   0.846234  0.812752  2.07632      1.28616  1.1478    1.35084
 0.740079  3.25956   1.11199   1.12065      2.44196  2.97778   0.823054
 3.05069   2.62497   1.50005   1.41386      3.28416  1.12238   2.214
 1.26255   2.59351   5.15911   0.718461     2.29101  1.07196   1.32316
 1.06029   0.193651  5.10846   1.49302   …  1.1329   0.841168  1.67932
 2.89129   1.37615   2.85749   1.36728      1.99558  3.09876   0.870452
 2.13142   1.73405   1.6878    1.32262      1.53206  3.41373   2.45196
 5.38291   1.30888   3.09052   0.559428     1.65195  4.15913   2.07091

In [7]:
# flatten into 3D matrix
X = reshape(hcat(xs...), nx, S, B)
Y = reshape(hcat(ys...), ny, S, B)
Y[:,:,2]

9×4300 Matrix{Float32}:
 0.918382  1.17214   1.58792   1.78473   …  1.57903  0.909114  1.3323
 5.10085   0.846234  0.812752  2.07632      1.28616  1.1478    1.35084
 0.740079  3.25956   1.11199   1.12065      2.44196  2.97778   0.823054
 3.05069   2.62497   1.50005   1.41386      3.28416  1.12238   2.214
 1.26255   2.59351   5.15911   0.718461     2.29101  1.07196   1.32316
 1.06029   0.193651  5.10846   1.49302   …  1.1329   0.841168  1.67932
 2.89129   1.37615   2.85749   1.36728      1.99558  3.09876   0.870452
 2.13142   1.73405   1.6878    1.32262      1.53206  3.41373   2.45196
 5.38291   1.30888   3.09052   0.559428     1.65195  4.15913   2.07091

In [8]:
function save_ci_results(results, methodId, B, S, pooled=nothing, two_sided=nothing; prefix="", dir="./")
    averages = []

    for batchId in 1:B
        batch = results[methodId, batchId, :]
        coverage = sum([j for (j, _) in batch]) / S
        width = sum([j for (_, j) in batch]) / S
        push!(averages, (coverage, width))
    end
    if isnothing(two_sided)
        save(averages, distrX[1:B], distrY[1:B], alpha, prefix=prefix, dir=dir)
    else
        alpha_ = two_sided ? alpha : alpha / 2
        save(averages, distrX[1:B], distrY[1:B], alpha_, pooled, two_sided, prefix=prefix, dir=dir)
    end
end

function save_permutation_results(results, B, S; prefix="", dir="./")
    i = 1
    per_method = []
    for two_sided in [true, false]
        for pooled in [true, false]
            save_ci_results(results, i, B, S, pooled, two_sided, prefix=prefix, dir=dir)
            i += 1
        end
    end
end

save_permutation_results (generic function with 1 method)

In [9]:
T = Threads.nthreads()
results = Array{Union{Tuple, Nothing}, 3}(nothing, 6, B, S)

#@time Threads.@threads for (i,j) in collect(Iterators.product(1:B, 1:S)) # 15.52 sec on (B,S) = (5, 1800)
@time @floop ThreadedEx(basesize=div(B*S, T)) for b in 1:B, s in 1:S
    @inbounds x = X[:,s,b]
    @inbounds y = Y[:,s,b]
 
    results[1, b, s] = permInterval(x, y, deltas[b], perms, true, alpha, twoSided, twoSided, 0.0005)    
    results[2, b, s] = permInterval(x, y, deltas[b], perms, false, alpha, twoSided, twoSided, 0.0005)
    results[3, b, s] = permInterval(x, y, deltas[b], perms, true, alpha/2, greater, smaller, 0.0005)
    results[4, b, s] = permInterval(x, y, deltas[b], perms, false, alpha/2, greater, smaller, 0.0005)
    
    """
    results[5, b, s] = bootstrap(x, y, deltas[b], alpha, nsamples=10_000)
    results[6, b, s] = tconf(x, y, deltas[b], alpha, false)
    """
end

dir = "../results/" * string(nx) * "_" * string(ny) * "/1/"
save_permutation_results(results, B, S; dir=dir)
#save_ci_results(results, 6, B, S; prefix="bs", dir=dir)

3846.562758 seconds (3.33 G allocations: 78.381 TiB, 51.52% gc time, 0.16% compilation time)
dir * filename = "../results/8_9/1/twoSided_pooled_0.05.csv"
dir * filename = "../results/8_9/1/twoSided_unpooled_0.05.csv"
dir * filename = "../results/8_9/1/oneSided_pooled_0.025.csv"
dir * filename = "../results/8_9/1/oneSided_unpooled_0.025.csv"
